In [1]:
from selenium.webdriver.common.by import By 
import undetected_chromedriver as uc 
import json
import time
import random

In [2]:
keys = {'color': 'ad', 'fuelTypeId': 'ad', 'fuelType': 'leadData', 'id': 'ad', 'km': 'ad', 'make': 'ad', 'makeId': 'ad', 'model': 'ad'
                    , 'modelId': 'ad', 'price': 'ad', 'province': 'ad', 'provinceId': 'ad', 'title': 'ad', 'transmissionType': 'ad', 'transmissionTyId': 'ad', 'year': 'ad'
                    , 'brand': 'leadData', 'brand_id': 'leadData', 'fuel_type': 'ad', 'km': 'leadData', 'model': 'leadData', 'model_id': 'leadData', 'price': 'leadData'
                    , 'seller_type': 'leadData', 'bodyType': 'vehicleInfo/specifications', 'cubicCapacity': 'vehicleInfo/specifications', 'doors': 'vehicleInfo/specifications'
                    , 'hp': 'vehicleInfo/specifications'}

def get_only_fisrt_value(keys_and_values):
    for key in keys_and_values:
        datas = keys_and_values[key]
        if len(datas) == 0:
            keys_and_values[key] = ''
        else:
            keys_and_values[key] = keys_and_values[key][0]
    return keys_and_values
            
def search_all_coincidences(key, path, data_json):
    coincidences = []
    data_json = get_sub_path(path, data_json)
    if isinstance(data_json, list):
        for data in data_json:
            if isinstance(data, dict) or isinstance(data, list):
                coincidences = coincidences + search_all_coincidences(key, None, data)
    else:
        for data_key in data_json:
            data = data_json[data_key]
            if key == data_key:
                coincidences.append(data)
            elif isinstance(data, dict) or isinstance(data, list):
                coincidences = coincidences + search_all_coincidences(key, None, data)
    return coincidences
            
def get_path(keys, key):
    path = None
    if isinstance(keys, dict):
        path = keys[key]
    return path

def get_sub_path(path, data_json):
    if path != None:
        path = path.split('/')
        for sub_path in path:
            data_json = data_json[sub_path]
    return data_json

def get_data_in_dictionary(scriptText):
    if scriptText.startswith('window.__INITIAL_PROPS__ = JSON.parse("'):
        initialPos = scriptText.find('parse("') + 7
        lastPos = len(scriptText) - 3
        searchStringAd = scriptText[initialPos : lastPos]
        searchStringAd=searchStringAd.replace('\\\\\\\"', '')
        searchStringAd=searchStringAd.replace('\\\"', '\"')
        return json.loads(searchStringAd)

In [ ]:
driver = uc.Chrome(version_main=97)
list_link=[]
user_agents = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36',
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:96.0) Gecko/20100101 Firefox/96.0',
                'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
                'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36',
                'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:24.0) Gecko/20100101 Firefox/24.0']
agents = len(user_agents) - 1

pages = ['https://www.coches.net/segunda-mano/?MakeId=28&MaxYear=2022&MinKms=10000&MinYear=2010', 
         'https://www.coches.net/segunda-mano/?MakeId=28&MaxYear=2022&MinKms=10000&MinYear=2010&pg=2',
         'https://www.coches.net/segunda-mano/?MakeId=28&MaxYear=2022&MinKms=10000&MinYear=2010&pg=3',
         'https://www.coches.net/segunda-mano/?MakeId=28&MaxYear=2022&MinKms=10000&MinYear=2010&pg=4',
         'https://www.coches.net/segunda-mano/?MakeId=28&MaxYear=2022&MinKms=10000&MinYear=2010&pg=5',
         'https://www.coches.net/segunda-mano/?MakeId=28&MaxYear=2022&MinKms=10000&MinYear=2010&pg=6',
         'https://www.coches.net/segunda-mano/?MakeId=28&MaxYear=2022&MinKms=10000&MinYear=2010&pg=7',
         'https://www.coches.net/segunda-mano/?MakeId=28&MaxYear=2022&MinKms=10000&MinYear=2010&pg=8',
         'https://www.coches.net/segunda-mano/?MakeId=28&MaxYear=2022&MinKms=10000&MinYear=2010&pg=9',
         'https://www.coches.net/segunda-mano/?MakeId=28&MaxYear=2022&MinKms=10000&MinYear=2010&pg=10',
         'https://www.coches.net/segunda-mano/?MakeId=30&MaxYear=2022&MinKms=10000&MinYear=2010',
         'https://www.coches.net/segunda-mano/?MakeId=30&MaxYear=2022&MinKms=10000&MinYear=2010&pg=2',
         'https://www.coches.net/segunda-mano/?MakeId=30&MaxYear=2022&MinKms=10000&MinYear=2010&pg=3',
         'https://www.coches.net/segunda-mano/?MakeId=30&MaxYear=2022&MinKms=10000&MinYear=2010&pg=4',
         'https://www.coches.net/segunda-mano/?MakeId=25&MaxYear=2022&MinKms=10000&MinYear=2010',
         'https://www.coches.net/segunda-mano/?MakeId=25&MaxYear=2022&MinKms=10000&MinYear=2010&pg=2',
         'https://www.coches.net/segunda-mano/?MakeId=25&MaxYear=2022&MinKms=10000&MinYear=2010&pg=3',
         'https://www.coches.net/segunda-mano/?MakeId=25&MaxYear=2022&MinKms=10000&MinYear=2010&pg=4'
         ]
first = True

for page in pages:
    increment = 100
    agent_number = random.randint(0, agents)
    driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": user_agents[agent_number]})
    driver.get(page)
    if first:
        button = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div[2]/div/div/div/footer/div/button[2]")
        button.click()
        first = False
    while True:
        driver.execute_script("window.scrollTo(0, " + str(increment) + ");")
        time.sleep(0.2)
        elements = driver.find_elements(By.CLASS_NAME, 'mt-CardBasic-titleLink')
        increment += 200
        if len(elements) >= 30 or increment >= 6000:
            break
        time.sleep(random.randint(3, 5))
        
    for e in elements:
        link=e.get_attribute("href")
        print(link)
        list_link.append(link)
        
    

list_restult=[]
for l in list_link:
    if l not in list_restult:
        list_restult.append(l)


In [23]:
coches = [
"https://www.coches.net/mitsubishi-asx-160-mpi-motion-5p-gasolina-2014-en-malaga-51355202-covo.aspx",
"https://www.coches.net/mitsubishi-asx-160-mpi-motion-5p-gasolina-2019-en-alicante-51355008-covo.aspx",
"https://www.coches.net/mitsubishi-asx-160-mpi-motion-5p-gasolina-2018-en-malaga-51354996-covo.aspx",
"https://www.coches.net/mitsubishi-asx-160-mpi-motion-5p-gasolina-2019-en-alicante-51354930-covo.aspx",
"https://www.coches.net/mitsubishi-eclipse-cross-150t-motion-cvt-5p-gasolina-2019-en-alicante-51354882-covo.aspx",
"https://www.coches.net/mitsubishi-asx-160-mpi-challenge-5p-gasolina-2019-en-valencia-51016899-covo.aspx",
"https://www.coches.net/mitsubishi-asx-180-did-kaiteki-5p-diesel-2014-en-zaragoza-51354557-covo.aspx",
"https://www.coches.net/mitsubishi-asx-160-mpi-motion-5p-gasolina-2018-en-zaragoza-51354456-covo.aspx",
"https://www.coches.net/mitsubishi-asx-200-mpi-cvt-kaiteki-5p-gasolina-2020-en-las_palmas-51224856-covo.aspx",
"https://www.coches.net/mitsubishi-l200-cc-220-did-mpro-4p-diesel-2020-en-sevilla-50930639-covo.aspx",
"https://www.coches.net/mitsubishi-asx-200-did-challenge-4wd-5p-diesel-2014-en-huelva-50892209-covo.aspx",
"https://www.coches.net/lexus-es-25-300h-business-4p-electrico-hibrido-2020-en-madrid-50620587-covo.aspx",
"https://www.coches.net/lexus-is-300h-executive-tecno--navibox-4p-electrico-hibrido-2015-en-jaen-42524549-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-business-navigation-2wd-5p-electrico-hibrido-2018-en-barcelona-51261198-covo.aspx",
"https://www.coches.net/lexus-ct-18-200h-executive-otros-2018-en-madrid-50979878-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-executive-4wd-5p-electrico-hibrido-2018-en-barcelona-51093735-covo.aspx",
"https://www.coches.net/lexus-is-300h-executive-tecno-4p-electrico-hibrido-2015-en-madrid-51349008-covo.aspx",
"https://www.coches.net/lexus-ct-18-200h-executive-5p-electrico-hibrido-2018-en-madrid-51202622-covo.aspx",
"https://www.coches.net/lexus-is-25-300h-executive-tecno--navibox-4p-electrico-hibrido-2015-en-valencia-51295606-covo.aspx",
"https://www.coches.net/lexus-is-25-300h-eco-otros-2017-en-sevilla-51366448-covo.aspx",
"https://www.coches.net/lexus-ct-200h-executive-5p-electrico-hibrido-2015-en-sevilla-51365977-covo.aspx",
"https://www.coches.net/lexus-nx-300h-executive-4wd--navibox-5p-electrico-hibrido-2015-en-tarragona-51160992-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-business-2wd-5p-electrico-hibrido-2018-en-valencia-50795289-covo.aspx",
"https://www.coches.net/lexus-is-25-300h-eco-4p-electrico-hibrido-2020-en-barcelona-51013136-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-business-navigation-2wd-5p-electrico-hibrido-2017-en-madrid-51365366-covo.aspx",
"https://www.coches.net/lexus-is300--5p-otros-2017-en-madrid-51367080-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-eco-2wd-5p-electrico-hibrido-2016-en-madrid-51052879-covo.aspx",
"https://www.coches.net/lexus-ux-20-250h-business-5p-electrico-hibrido-2020-en-leon-50528955-covo.aspx",
"https://www.coches.net/lexus-ct-18-200h-luxury-5p-electrico-hibrido-2018-en-madrid-51112862-covo.aspx",
"https://www.coches.net/lexus-rc-25-rc-300h-f-sport-2p-electrico-hibrido-2017-en-madrid-50855266-covo.aspx",
"https://www.coches.net/lexus-ux-20-250h-business-5p-electrico-hibrido-2019-en-vizcaya-51351667-covo.aspx",
"https://www.coches.net/lexus-nx-300h-corporate-2wd-5p-electrico-hibrido-2015-en-sevilla-51140041-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-f-sport-4wd-panoramico-5p-electrico-hibrido-2017-en-vizcaya-51363629-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-business-navigation-2wd-5p-electrico-hibrido-2020-en-valencia-50338520-covo.aspx",
"https://www.coches.net/lexus-ux-20-250h-executive-4wd-5p-electrico-hibrido-2019-en-barcelona-49266758-covo.aspx",
"https://www.coches.net/lexus-rc-25-rc-300h-f-sport-2p-electrico-hibrido-2016-en-barcelona-51363010-covo.aspx",
"https://www.coches.net/lexus-is-200d-luxury-4p-diesel-2012-en-alicante-51317767-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-executive-4wd-5p-electrico-hibrido-2018-en-madrid-51035201-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-business-2wd-5p-electrico-hibrido-2018-en-vizcaya-50974782-covo.aspx",
"https://www.coches.net/lexus-nx-300h-corporate-2wd-5p-electrico-hibrido-2015-en-barcelona-51361959-covo.aspx",
"https://www.coches.net/lexus-ct-18-200h-executive-otros-2018-en-barcelona-51361913-covo.aspx",
"https://www.coches.net/lexus-gs-450h-hybrid-drive-otros-2013-en-sevilla-51361190-covo.aspx",
"https://www.coches.net/lexus-is-300h-executive-tecno--navibox-otros-2014-en-sevilla-51361153-covo.aspx",
"https://www.coches.net/lexus-is-25-300h-executive-otros-2015-en-sevilla-51361106-covo.aspx",
"https://www.coches.net/lexus-is-300h-executive-tecno-otros-2014-en-sevilla-51361051-covo.aspx",
"https://www.coches.net/lexus-rx-450h-executive-tecno-5p-electrico-hibrido-2017-en-alicante-51017114-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-executive-navigation-5p-electrico-hibrido-2019-en-madrid-50467577-covo.aspx",
"https://www.coches.net/lexus-ls-500h-luxury-art-wood-lwhite-4p-electrico-hibrido-2018-en-burgos-47970725-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-corporate-2wd-5p-electrico-hibrido-2017-en-madrid-51308218-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-business-navigation-2wd-5p-electrico-hibrido-2019-en-madrid-51359595-covo.aspx",
"https://www.coches.net/lexus-rx-450h-luxury-5p-electrico-hibrido-2011-en-sevilla-51179714-covo.aspx",
"https://www.coches.net/lexus-rx-450h-l-executive-5p-electrico-hibrido-2019-en-guadalajara-51359226-covo.aspx",
"https://www.coches.net/lexus-es-25-300h-f-sport-4p-electrico-hibrido-2019-en-guadalajara-51359196-covo.aspx",
"https://www.coches.net/lexus-es-25-300h-business-4p-electrico-hibrido-2019-en-guadalajara-51359169-covo.aspx",
"https://www.coches.net/lexus-rx-450h-hybrid-f-sport-5p-electrico-hibrido-2013-en-guadalajara-51359108-covo.aspx",
"https://www.coches.net/lexus-gs-300h-executive-4p-electrico-hibrido-2016-en-navarra-49767302-covo.aspx",
"https://www.coches.net/lexus-is-25-300h-executive-4p-electrico-hibrido-2020-en-valladolid-51084200-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-executive-4wd--navibox-5p-electrico-hibrido-2017-en-a_coruna-51213878-covo.aspx",
"https://www.coches.net/lexus-rx-450h-hybrid-f-sport-5p-electrico-hibrido-2013-en-burgos-42765372-covo.aspx",
"https://www.coches.net/lexus-rx-450h-f-sport-5p-electrico-hibrido-2020-en-burgos-48666474-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-business-2wd-5p-electrico-hibrido-2018-en-a_coruna-51176340-covo.aspx",
"https://www.coches.net/lexus-rx-450h-premium-5p-electrico-hibrido-2010-en-valladolid-51358433-covo.aspx",
"https://www.coches.net/lexus-rx-450h-hybrid-f-sport-5p-electrico-hibrido-2013-en-burgos-50316143-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-sport-edition-2wd-5p-electrico-hibrido-2017-en-madrid-51358196-covo.aspx",
"https://www.coches.net/lexus-gs-450h-hybrid-drive-otros-2014-en-alicante-51357984-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-business-2wd-5p-electrico-hibrido-2020-en-valencia-51180872-covo.aspx",
"https://www.coches.net/lexus-is-25-300h-executive-4p-electrico-hibrido-2015-en-la_rioja-51357565-covo.aspx",
"https://www.coches.net/lexus-es-25-300h-business-4p-electrico-hibrido-2019-en-madrid-50934845-covo.aspx",
"https://www.coches.net/lexus-es-25-300h-f-sport-4p-electrico-hibrido-2019-en-madrid-50934844-covo.aspx",
"https://www.coches.net/lexus-nx-300h-corporate-2wd-5p-electrico-hibrido-2015-en-valencia-51202665-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-eco-2wd-5p-electrico-hibrido-2016-en-valencia-51180842-covo.aspx",
"https://www.coches.net/lexus-ct-18-200h-eco-5p-electrico-hibrido-2017-en-barcelona-51242315-covo.aspx",
"https://www.coches.net/lexus-nx-300h-corporate-2wd-5p-electrico-hibrido-2015-en-madrid-51356969-covo.aspx",
"https://www.coches.net/lexus-nx-300h-corporate-2wd-5p-electrico-hibrido-2015-en-valencia-51356859-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-executive-4wd-tecno-5p-electrico-hibrido-2016-en-asturias-50252830-covo.aspx",
"https://www.coches.net/lexus-ct-18-200h-business-5p-electrico-hibrido-2020-en-girona-51356739-covo.aspx",
"https://www.coches.net/lexus-is-25-300h-business-4p-electrico-hibrido-2018-en-barcelona-51356722-covo.aspx",
"https://www.coches.net/lexus-ct-18-200h-business-5p-electrico-hibrido-2020-en-granada-51356367-covo.aspx",
"https://www.coches.net/lexus-ct-18-200h-business-5p-electrico-hibrido-2020-en-alicante-51356276-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-business-2wd-5p-electrico-hibrido-2018-en-navarra-50977136-covo.aspx",
"https://www.coches.net/lexus-is-25-300h-business-4p-electrico-hibrido-2018-en-madrid-51263120-covo.aspx",
"https://www.coches.net/lexus-rx-450h-business-plus-5p-electrico-hibrido-2018-en-sevilla-51355823-covo.aspx",
"https://www.coches.net/lexus-rx-450h-l-executive-5p-electrico-hibrido-2020-en-alicante-51119837-covo.aspx",
"https://www.coches.net/lexus-gs-450h-president-hibrido-4p-electrico-hibrido-2010-en-sevilla-48423434-covo.aspx",
"https://www.coches.net/lexus-rc-25-300h-executive-navigation-2p-electrico-hibrido-2018-en-badajoz-50706305-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-business-navigation-2wd-5p-electrico-hibrido-2019-en-madrid-51355409-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-premium-plus-4wd-5p-electrico-hibrido-2021-en-leon-51355410-covo.aspx",
"https://www.coches.net/lexus-ct-200h-fsport--cuero-5p-electrico-hibrido-2014-en-vizcaya-51293952-covo.aspx",
"https://www.coches.net/lexus-is-25-300h-executive-4p-electrico-hibrido-2018-en-sevilla-51082774-covo.aspx",
"https://www.coches.net/lexus-is-25-300h-business-4p-electrico-hibrido-2018-en-malaga-51354826-covo.aspx",
"https://www.coches.net/lexus-ux-20-250h-business-navigation-otros-2020-en-sevilla-51247077-covo.aspx",
"https://www.coches.net/lexus-rx-450h-luxury-otros-2015-en-sevilla-51209264-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-eco-2wd-otros-2016-en-sevilla-51247421-covo.aspx",
"https://www.coches.net/lexus-ct-18-200h-executive-tecno-otros-2016-en-barcelona-51090768-covo.aspx",
"https://www.coches.net/lexus-is-25-300h-business-4p-electrico-hibrido-2018-en-zaragoza-51354397-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-executive-4wd-5p-electrico-hibrido-2018-en-madrid-51354290-covo.aspx",
"https://www.coches.net/lexus-ux-20-250h-executive-5p-electrico-hibrido-2019-en-las_palmas-51224927-covo.aspx",
"https://www.coches.net/lexus-ct-18-200h-eco-5p-otros-2018-en-navarra-50756201-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-executive-4wd-5p-electrico-hibrido-2017-en-madrid-51261671-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-executive-4wd-5p-electrico-hibrido-2018-en-madrid-51273991-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-business-navigation-2wd-5p-electrico-hibrido-2018-en-madrid-51261672-covo.aspx",
"https://www.coches.net/lexus-rc-25-rc-300h-executive-2p-electrico-hibrido-2017-en-barcelona-50945001-covo.aspx",
"https://www.coches.net/lexus-gs-300h-executive-4p-electrico-hibrido-2016-en-vizcaya-48118291-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-premium-2wd-otros-2021-en-alava-50647939-covo.aspx",
"https://www.coches.net/lexus-ux-20-250h-executive-5p-electrico-hibrido-2019-en-las_palmas-51283015-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-business-navigation-2wd-5p-electrico-hibrido-2019-en-madrid-51352003-covo.aspx",
"https://www.coches.net/lexus-lc-35-v6-500h-sport-2p-electrico-hibrido-2019-en-madrid-51170887-covo.aspx",
"https://www.coches.net/lexus-nx-300h-corporate-2wd-5p-electrico-hibrido-2015-en-navarra-51351827-covo.aspx",
"https://www.coches.net/lexus-es-25-300h-executive-4p-electrico-hibrido-2019-en-madrid-51178474-covo.aspx",
"https://www.coches.net/lexus-es-25-300h-executive-4p-electrico-hibrido-2019-en-madrid-51274779-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-corporate-2wd-5p-electrico-hibrido-2016-en-madrid-47670524-covo.aspx",
"https://www.coches.net/lexus-ct-18-200h-business-otros-2018-en-malaga-51351776-covo.aspx",
"https://www.coches.net/lexus-rc-25-300h-executive-2p-electrico-hibrido-2017-en-madrid-47670354-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-business-2wd-5p-electrico-hibrido-2017-en-madrid-50595985-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-executive-4wd--navibox-5p-electrico-hibrido-2017-en-pontevedra-51351639-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-executive-navigation-4wd-5p-electrico-hibrido-2018-en-pontevedra-51351633-covo.aspx",
"https://www.coches.net/lexus-nx-300h-corporate-2wd-5p-electrico-hibrido-2015-en-malaga-51351578-covo.aspx",
"https://www.coches.net/lexus-rx-450h-business-5p-electrico-hibrido-2018-en-madrid-51131597-covo.aspx",
"https://www.coches.net/lexus-rc-f-carbon-package-2p-gasolina-2015-en-madrid-51196378-covo.aspx",
"https://www.coches.net/lexus-rc-f-carbon-package-2p-gasolina-2015-en-madrid-51196317-covo.aspx",
"https://www.coches.net/lexus-rc-f-carbon-package-2p-gasolina-2015-en-madrid-51196289-covo.aspx",
"https://www.coches.net/lexus-rc-f-carbon-package-2p-gasolina-2015-en-madrid-51196255-covo.aspx",
"https://www.coches.net/lexus-rc-50-v8-f-carbon-package-2p-gasolina-2019-en-madrid-51240287-covo.aspx",
"https://www.coches.net/lexus-rc-25-rc-300h-executive-2p-electrico-hibrido-2016-en-madrid-51283070-covo.aspx",
"https://www.coches.net/lexus-rc-25-300h-luxury-2p-electrico-hibrido-2018-en-madrid-50521256-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-luxury-4wd-5p-electrico-hibrido-2020-en-madrid-51240299-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-f-sport-panoramico-4wd-5p-electrico-hibrido-2021-en-madrid-51324472-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-f-sport-panoramico-4wd-5p-electrico-hibrido-2020-en-madrid-50797301-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-f-sport-panoramico-4wd-5p-electrico-hibrido-2020-en-madrid-50347433-covo.aspx",
"https://www.coches.net/lexus-nx-25-300h-executive-navigation-4wd-5p-electrico-hibrido-2018-en-madrid-51240239-covo.aspx",
"https://www.coches.net/lexus-gs-300h-executive-4p-electrico-hibrido-2017-en-zaragoza-50959185-covo.aspx"]

In [24]:
driver = uc.Chrome(version_main=97)
keys_and_values = {}
contador = 1

for url in coches:
    driver.get(url)
    
    print(contador)
    time.sleep(5)

    content = driver.page_source
    scripts = driver.find_element_by_xpath("/html/body/script[3]")
    scripts = scripts.get_attribute("innerHTML")

    data_json = get_data_in_dictionary(scripts)
    for key in keys:
        path = get_path(keys, key)
        keys_and_values[key] = search_all_coincidences(key, path, data_json)
    keys_and_values = get_only_fisrt_value(keys_and_values)
    if bool(keys_and_values):
        with open('coche.json', 'a') as fp:
            json.dump(keys_and_values, fp)
    contador +=1

1


/tmp/ipykernel_3814/156695305.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  scripts = driver.find_element_by_xpath("/html/body/script[3]")


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131


KeyError: 'ad'